In [1]:
import os

import findspark
findspark.init("/home/pc/TestJupyter/opt/spark-3.3.0/spark-3.3.0-bin-hadoop3")
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import HiveContext
from pyspark.sql import SparkSession
from pyspark.sql import Row


# warehouse_location points to the default location for managed databases and tables
os.environ["PYSPARK_PYTHON"]="/home/pc/TestJupyter/opt/spark-3.3.0/venv-spark/bin/python39"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.12:3.3.0  pyspark-shell'

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.driver.memory", "75g")\
    .config("spark.sql.warehouse.dir", "hdfs://10.123.51.78:8000/user/hive/warehouse") \
    .config("spark.sql.catalogImplementation", "hive")\
    .config("hive.metastore.uris", "thrift://g2.bigtop.it:9083")\
    .enableHiveSupport() \
    .getOrCreate()

def read_avro(path):
    return spark.read.format("avro").load(path)

def write_avro(df, path):
    return df.write.format("avro").save(path)

:: loading settings :: url = jar:file:/home/pc/TestJupyter/opt/spark-3.3.0/spark-3.3.0-bin-hadoop3/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pc/.ivy2/cache
The jars for the packages stored in: /home/pc/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-76bc52c4-3280-4900-8222-b3c17769544b;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.3.0 in central
	found org.tukaani#xz;1.8 in central
	found org.spark-project.spark#unused;1.0.0 in central
:: resolution report :: resolve 333ms :: artifacts dl 34ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.3.0 from central in [default]
	org.spark-project.spark#unused;1.0.0 from central in [default]
	org.tukaani#xz;1.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   

22/10/08 03:34:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
def q(query):
    return spark.sql(query)

In [3]:
q("show tables;").show(100, False)

+---------+---------------------------+-----------+
|namespace|tableName                  |isTemporary|
+---------+---------------------------+-----------+
|default  |books_ext                  |false      |
|default  |source_data                |false      |
|default  |tag_gram_ranking           |false      |
|default  |tag_gram_ranking_small     |false      |
|default  |transaction_detail_hive_tbl|false      |
|default  |wiki_gram                  |false      |
+---------+---------------------------+-----------+



In [36]:
df = read_avro("output/25_CENTER_DETERMINED")
df = df.repartition(1200)

In [4]:
bmData = df.filter(df.language == "malay")
bmData.show()

+------------+--------------------+--------------------+--------+--------------------+----------+---+------------------+----------+---------+-----------+-----------+----------+
| sentence_id|            original|                 tag|language|               token|normal_tag|pos|        token_gram|  tag_gram|gram_type|  unique_id|containsKey|middle_key|
+------------+--------------------+--------------------+--------+--------------------+----------+---+------------------+----------+---------+-----------+-----------+----------+
|446676611706|răsuceni merupaka...|[PROPN, VERB, DET...|   malay|[rasuceni, merupa...|      null|  3|        komun yang| NOUN PRON|        2|68720377715|       true|     false|
| 85899359585|sekolah kebangsaa...|[NOUN, PROPN, PRO...|   malay|[sekolah, kebangs...|      null|  3|         barat sek| NOUN NOUN|        2|25770715767|       true|     false|
|214748377137|sejumlah kuasa te...|[DET, NOUN, ADV, ...|   malay|[sejumlah, kuasa,...|      null| 15|        barat 

In [5]:
biData = df.filter(df.language == "english")
biData.show()

+------------+--------------------+--------------------+--------+--------------------+----------+---+--------------------+----------+---------+-----------+-----------+----------+
| sentence_id|            original|                 tag|language|               token|normal_tag|pos|          token_gram|  tag_gram|gram_type|  unique_id|containsKey|middle_key|
+------------+--------------------+--------------------+--------+--------------------+----------+---+--------------------+----------+---------+-----------+-----------+----------+
|274877917960|englewood high sc...|[VERB, ADJ, NOUN,...| english|[englewood, high,...|      null|  6|  school unification| NOUN NOUN|        2|77309669822|       true|     false|
|481036347329|tyagaraja aradhan...|[NOUN, NOUN, NOUN...| english|[tyagaraja, aradh...|      null| 18|          school run| NOUN VERB|        2|68719716311|       true|     false|
| 85899354447|the cultural and ...|[DET, ADJ, CCONJ,...| english|[the, cultural, a...|      null| 19|    

In [37]:
bcData = df.filter(df.language == "chinese")
bcData.show()

+------------+------------------------------------+--------------------+--------+------------------------------+--------------------+---+-------------+--------+---------+-----------+-----------+----------+
| sentence_id|                            original|                 tag|language|                         token|          normal_tag|pos|   token_gram|tag_gram|gram_type|  unique_id|containsKey|middle_key|
+------------+------------------------------------+--------------------+--------+------------------------------+--------------------+---+-------------+--------+---------+-----------+-----------+----------+
|438086670324|年月日 夸耶和加纳球队十一智者签约...|[t, nrt, c, v, n,...| chinese|  [年月日, 夸耶, 和, 加纳, ...|[t, NOUN, CONJ, V...| 50|    中国 联赛|   ns vn|        2|68719927430|       true|     false|
|197568498086| 江安世 台湾生物学家 中央研究院院...|[ns, n, ns, n, nt...| chinese| [江安, 世, 台湾, 生物学家,...|[PROPN, NOUN, PRO...|  2|台湾 生物学家|    ns n|        2|17180234583|       true|     false|
| 85899348597|不过 有一些反对倪柝声神学立

In [8]:
bitokenFreq = biData.groupBy("token_gram").count()
bmtokenFreq = bmData.groupBy("token_gram").count()
bctokenFreq = bcData.groupBy("token_gram").count()


In [9]:
bitokenFreq.show()

+--------------------+-----+
|          token_gram|count|
+--------------------+-----+
|           school is| 4128|
|       school by the|   93|
|     school for chef|    1|
|college of the ma...|    9|
|   school of theater|   15|
|    school limestone|    1|
|          school for| 4284|
|    school attending|    9|
|university has so...|    1|
|  school wrestler he|    1|
|university elemen...|    5|
| college were raided|    1|
|university of chi...|   15|
|          school has| 1142|
|     school building|  779|
| college of business|  292|
|college western u...|    1|
|college which com...|    1|
|school where he d...|    2|
|school marquardt ...|    1|
+--------------------+-----+
only showing top 20 rows



In [9]:
import shutil
shutil.rmtree("output/27_BM_TOKEN_F")
write_avro(bmtokenFreq, "output/27_BM_TOKEN_F")

In [11]:
write_avro(bitokenFreq, "output/27_BI_TOKEN_F")

In [13]:
import shutil
shutil.rmtree("output/27_BC_TOKEN_F")
write_avro(bctokenFreq, "output/27_BC_TOKEN_F")

In [4]:
bctokF = read_avro("output/27_BC_TOKEN_F")
bmtokF = read_avro("output/27_BM_TOKEN_F")
bitokF = read_avro("output/27_BI_TOKEN_F")

bctokF = bctokF.repartition(800)
bmtokF = bmtokF.repartition(800)
bitokF = bitokF.repartition(800)




In [5]:
bcwithTokF = bcData.join(bctokF, bcData.token_gram == bctokF.token_gram).drop(bctokF.token_gram)
biwithTokF = biData.join(bitokF, biData.token_gram == bitokF.token_gram).drop(bitokF.token_gram)
bmwithTokF = bmData.join(bmtokF, bmData.token_gram == bmtokF.token_gram).drop(bmtokF.token_gram)

NameError: name 'bcData' is not defined

In [9]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [10]:
withTokF = unionAll(bcwithTokF, biwithTokF, bmwithTokF)

In [11]:
import shutil
shutil.rmtree("output/27_TokFreq")
write_avro(withTokF, "output/27_TokFreq")

In [ ]:
## tag freq

In [66]:
df = read_avro("output/27_TokFreq")
df = df.repartition(1800)
# tokenFreq = df.groupBy("token_gram").count()
df = df.withColumnRenamed("count", "tok_gram_f")

In [67]:
bmData = df.filter(df.language == "malay")
bmData.show()

+------------+--------------------+--------------------+--------+--------------------+----------+---+--------------------+--------------------+---------+------------+-----------+----------+----------+
| sentence_id|            original|                 tag|language|               token|normal_tag|pos|          token_gram|            tag_gram|gram_type|   unique_id|containsKey|middle_key|tok_gram_f|
+------------+--------------------+--------------------+--------+--------------------+----------+---+--------------------+--------------------+---------+------------+-----------+----------+----------+
|146028901054|kampung petani ia...|[NOUN, NOUN, AUX,...|   malay|[kampung, petani,...|      null|  6|       daerah tutong|           NOUN NOUN|        2|103079506608|       true|     false|        65|
|343597397391|sekolah kebangsaa...|[NOUN, NOUN, NOUN...|   malay|[sekolah, kebangs...|      null|  0|sekolah kebangsaa...|NOUN NOUN NOUN NO...|        5|652835290019|       true|     false|       

In [68]:
bcData = df.filter(df.language == "chinese")
bcData.show()

+------------+-------------------------------------+--------------------+--------+------------------------------+--------------------+---+--------------------------------+-----------+---------+------------+-----------+----------+----------+
| sentence_id|                             original|                 tag|language|                         token|          normal_tag|pos|                      token_gram|   tag_gram|gram_type|   unique_id|containsKey|middle_key|tok_gram_f|
+------------+-------------------------------------+--------------------+--------+------------------------------+--------------------+---+--------------------------------+-----------+---------+------------+-----------+----------+----------+
|335007449289|                 ian mccabe pat fo...|[eng, eng, eng, e...| chinese|          [ian, mccabe, pat...|[X, X, X, X, X, X...| 39|                    香港 和 中国|    ns c ns|        3|309238227534|       true|     false|        77|
|360777253795| 年 中华民国政府成立海军海道测量局...|[m, n

In [69]:
biData = df.filter(df.language == "english")
biData.show()

+------------+--------------------+--------------------+--------+--------------------+----------+---+--------------------+--------------------+---------+------------+-----------+----------+----------+
| sentence_id|            original|                 tag|language|               token|normal_tag|pos|          token_gram|            tag_gram|gram_type|   unique_id|containsKey|middle_key|tok_gram_f|
+------------+--------------------+--------------------+--------+--------------------+----------+---+--------------------+--------------------+---------+------------+-----------+----------+----------+
|103079220921|in he received an...|[ADP, PRON, VERB,...| english|[in, he, received...|      null| 10|university which ...|  NOUN PRON ADV VERB|        4|438086790373|       true|     false|        10|
|300647721484|the four elementa...|[DET, NUM, NOUN, ...| english|[the, four, eleme...|      null| 15|school and thomas...|NOUN CCONJ PROPN ...|        5|532576166304|       true|     false|       

In [11]:
bmFreq = bmData.groupBy("tag_gram").count().withColumnRenamed("count", "tag_gram_f")
bcFreq = bcData.groupBy("tag_gram").count().withColumnRenamed("count", "tag_gram_f")
biFreq = biData.groupBy("tag_gram").count().withColumnRenamed("count", "tag_gram_f")

In [12]:
write_avro(bmFreq, "output/bmFreq")

In [13]:
write_avro(biFreq, "output/biFreq")

In [14]:
write_avro(bcFreq, "output/bcFreq")

In [70]:
read_avro("output/bcFreq").show()

+-------------+----------+
|     tag_gram|tag_gram_f|
+-------------+----------+
| ns n n uj vn|       191|
|ns uj r vn ns|         1|
|    ns m n ns|       115|
|   ns ns ns f|       434|
|  ns v v nz m|         4|
|   ns f v v b|         1|
|    ns n n vn|      2151|
|       ns l r|        31|
|      ns j uj|        56|
|       ns m d|       318|
|   ns m b n v|        24|
|       ns v f|       699|
|   ns s ns vg|         5|
|ns nz vn m uj|         1|
|   ns nz ns n|       245|
|   ns n n v t|       127|
|ns n uj ns ns|      3217|
|   ns n f v p|        31|
|    ns v zg n|       797|
|      ns c ns|      3167|
+-------------+----------+
only showing top 20 rows



In [15]:
biData

DataFrame[sentence_id: bigint, original: string, tag: array<string>, language: string, token: array<string>, normal_tag: array<string>, pos: int, token_gram: string, tag_gram: string, gram_type: string, unique_id: bigint, containsKey: boolean, middle_key: boolean, tok_gram_f: bigint]

In [63]:
tfBM = read_avro("output/bmFreq")

In [64]:
withtagFBM = bmData.join(tfBM, bmData.tag_gram == tfBM.tag_gram).drop(tfBM.tag_gram)

In [65]:
withtagFBM

DataFrame[sentence_id: bigint, original: string, tag: array<string>, language: string, token: array<string>, normal_tag: array<string>, pos: int, token_gram: string, tag_gram: string, gram_type: string, unique_id: bigint, containsKey: boolean, middle_key: boolean, tok_gram_f: bigint, tag_gram_f: bigint]

In [22]:
write_avro(withtagFBM, "output/withTagFBM")

In [23]:
withtagFBM.show()

+------------+--------------------+--------------------+--------+--------------------+----------+---+--------------------+--------------------+---------+------------+-----------+----------+----------+----------+
| sentence_id|            original|                 tag|language|               token|normal_tag|pos|          token_gram|            tag_gram|gram_type|   unique_id|containsKey|middle_key|tok_gram_f|tag_gram_f|
+------------+--------------------+--------------------+--------+--------------------+----------+---+--------------------+--------------------+---------+------------+-----------+----------+----------+----------+
|146028901054|kampung petani ia...|[NOUN, NOUN, AUX,...|   malay|[kampung, petani,...|      null|  6|       daerah tutong|           NOUN NOUN|        2|103079506608|       true|     false|        65|     53113|
|343597397391|sekolah kebangsaa...|[NOUN, NOUN, NOUN...|   malay|[sekolah, kebangs...|      null|  0|sekolah kebangsaa...|NOUN NOUN NOUN NO...|        5

In [79]:
tfBI = read_avro("output/biFreq")

In [80]:
withtagFBI = biData.join(tfBI, biData.tag_gram == tfBI.tag_gram).drop(tfBI.tag_gram)

In [81]:
shutil.rmtree("output/withtagFBI")
write_avro(withtagFBI, "output/withtagFBI")

In [82]:
withtagFBI.show()

+------------+--------------------+--------------------+--------+--------------------+----------+---+--------------------+--------------------+---------+------------+-----------+----------+----------+----------+
| sentence_id|            original|                 tag|language|               token|normal_tag|pos|          token_gram|            tag_gram|gram_type|   unique_id|containsKey|middle_key|tok_gram_f|tag_gram_f|
+------------+--------------------+--------------------+--------+--------------------+----------+---+--------------------+--------------------+---------+------------+-----------+----------+----------+----------+
|103079220921|in he received an...|[ADP, PRON, VERB,...| english|[in, he, received...|      null| 10|university which ...|  NOUN PRON ADV VERB|        4|438086790373|       true|     false|        10|       372|
|300647721484|the four elementa...|[DET, NUM, NOUN, ...| english|[the, four, eleme...|      null| 15|school and thomas...|NOUN CCONJ PROPN ...|        5

In [83]:
tfBC = read_avro("output/bcFreq")

In [84]:
withtagFBC = bcData.join(tfBC, bcData.tag_gram == tfBC.tag_gram).drop(tfBC.tag_gram)

In [85]:
shutil.rmtree("output/withtagFBC")
write_avro(withtagFBC, "output/withtagFBC")

In [86]:
withtagFBC.show()

+------------+-------------------------------------+--------------------+--------+------------------------------+--------------------+---+--------------------------------+-----------+---------+------------+-----------+----------+----------+----------+
| sentence_id|                             original|                 tag|language|                         token|          normal_tag|pos|                      token_gram|   tag_gram|gram_type|   unique_id|containsKey|middle_key|tok_gram_f|tag_gram_f|
+------------+-------------------------------------+--------------------+--------+------------------------------+--------------------+---+--------------------------------+-----------+---------+------------+-----------+----------+----------+----------+
|335007449289|                 ian mccabe pat fo...|[eng, eng, eng, e...| chinese|          [ian, mccabe, pat...|[X, X, X, X, X, X...| 39|                    香港 和 中国|    ns c ns|        3|309238227534|       true|     false|        77|      316

In [58]:
bi

DataFrame[sentence_id: bigint, original: string, tag: array<string>, language: string, token: array<string>, normal_tag: array<string>, pos: int, token_gram: string, tag_gram: string, gram_type: string, unique_id: bigint, containsKey: boolean, middle_key: boolean, tok_gram_f: bigint, tag_gram_f: bigint]

In [59]:
bm

DataFrame[sentence_id: bigint, original: string, tag: array<string>, language: string, token: array<string>, normal_tag: array<string>, pos: int, token_gram: string, tag_gram: string, gram_type: string, unique_id: bigint, containsKey: boolean, middle_key: boolean, tok_gram_f: bigint, tag_gram_f: bigint]

In [60]:
bc

DataFrame[sentence_id: bigint, original: string, tag: array<string>, language: string, token: array<string>, normal_tag: array<string>, pos: int, token_gram: string, tag_gram: string, gram_type: string, unique_id: bigint, containsKey: boolean, middle_key: boolean, tag_gram_f: bigint]

In [87]:
bi = read_avro("output/withtagFBI")
bc = read_avro("output/withtagFBC")
bm = read_avro("output/withTagFBM")

In [88]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [89]:
tagGramFreq = unionAll(bi, bc, bm)

In [92]:
write_avro(tagGramFreq, "output/28_TagGramF")

In [95]:
import pyspark.sql.functions as F
read_avro("output/28_TagGramF").filter(F.col("language") == "english").show()

+------------+--------------------+--------------------+--------+--------------------+----------+---+--------------------+--------------------+---------+------------+-----------+----------+----------+----------+
| sentence_id|            original|                 tag|language|               token|normal_tag|pos|          token_gram|            tag_gram|gram_type|   unique_id|containsKey|middle_key|tok_gram_f|tag_gram_f|
+------------+--------------------+--------------------+--------+--------------------+----------+---+--------------------+--------------------+---------+------------+-----------+----------+----------+----------+
|163208764125|ucl has hundreds ...|[NOUN, VERB, NOUN...| english|[ucl, has, hundre...|      null| 32|university and th...| NOUN CCONJ DET NOUN|        4|489626419615|       true|     false|       272|      2574|
|403726932204|one known as coom...|[NUM, VERB, ADP, ...| english|[one, known, as, ...|      null| 32|         school home|           NOUN NOUN|        2